# Negative samples for the dataset

In [3]:
import pandas as pd
import numpy as np
import pydicom
import png
import math
from PIL import Image
from pathlib import Path
import boto3
import matplotlib.pyplot as plt

data_path = '/home/szelesteya/projects/EMBED_Open_Data/'
tables_path = data_path + 'tables/'
image_root_path = '/media/szelesteya/F824D4D024D492CC/EMBED-images/'
image_dcm_path = image_root_path + 'dicom-negative/'
image_png_path = image_root_path + 'negative-full'

In [4]:
cli_df = pd.read_csv(tables_path + 'EMBED_OpenData_clinical.csv', low_memory=False)

# Only keeping result BIRADS-1 and BIRADS-2 screenings
neg_cli_df = cli_df[(cli_df.asses.isin(['N'])) & 
                    cli_df.desc.str.contains('screen', case=False)][['Unnamed: 0',
                                                                     'empi_anon',
                                                                     'acc_anon',
                                                                     'side',
                                                                     'calcfind',
                                                                     'calcdistri',
                                                                     'otherfind',
                                                                     'numfind',
                                                                     'path_severity',
                                                                     'age_at_study',
                                                                     'ETHNICITY_DESC',
                                                                     'study_date_anon',
                                                                     'asses']]

In [5]:
# Merging clinical information with medical ones
meta_df = (pd.read_csv(tables_path + 'EMBED_OpenData_metadata_reduced.csv', low_memory=False))
meta_red_df = meta_df[(meta_df['spot_mag'] != 1) &
                      (meta_df['FinalImageType'] == '2D') &
                      (meta_df['ViewPosition'] == 'CC')][[  'empi_anon',
                                                            'acc_anon',
                                                            'ImageLateralityFinal',
                                                            'anon_dicom_path',
                                                            'ViewPosition',
                                                            'ROI_coords']]

# Rename columns to prepare for merge
meta_red_ren_df = meta_red_df.rename(columns={'ImageLateralityFinal':'side'})

In [6]:
neg_full_df = neg_cli_df.merge(meta_red_ren_df, on=['empi_anon','acc_anon','side'])

# Keeping relevant columns
neg_empi_df = neg_full_df[[ 'empi_anon',
                           'acc_anon',
                           'anon_dicom_path',
                           'side',
                           'asses',
                           'age_at_study',
                           'calcfind',
                           'calcdistri',
                           'otherfind',
                           'numfind',
                           'ROI_coords',
                           'ETHNICITY_DESC',
                           'study_date_anon']]

# Rename columns to be more consistent
neg_empi_df = neg_empi_df.rename(columns={'ETHNICITY_DESC':'eth_desc',
                                          'calcfind':'calc_find',
                                          'calcdistri':'calc_distrib',
                                          'otherfind':'other_find',
                                          'numfind':'num_find'})

# Generate paths for png extraction
neg_empi_df.loc[:, 'relative_dcm_path'] = neg_empi_df['anon_dicom_path'].apply(lambda x: '/'.join(x.split('/')[5:]))

In [7]:
# Ordering negative data by study date
neg_empi_ord_df = neg_empi_df.sort_values(by=['study_date_anon'], ascending=False)

# Choosing unique patients
neg_empi_fin_df = neg_empi_ord_df.drop_duplicates(subset=['empi_anon','acc_anon','side'], keep='first')[['empi_anon',
                                                                                       'acc_anon',
                                                                                       'side',
                                                                                       'asses',
                                                                                       'age_at_study',
                                                                                       'relative_dcm_path',
                                                                                       'calc_find',
                                                                                       'calc_distrib',
                                                                                       'other_find',
                                                                                       'num_find',
                                                                                       'ROI_coords',
                                                                                       'eth_desc',
                                                                                       'study_date_anon']]

In [2]:
# neg_empi_fin_df[(neg_empi_fin_df['ROI_coords'] == '()') & neg_empi_fin_df['num_find'] == 1]

NameError: name 'neg_empi_fin_df' is not defined

In [8]:
with open(data_path + 'negative_empirical.csv', 'w') as f:
    neg_empi_fin_df.to_csv(f, index=False)

In [9]:
with open(data_path + 'negative_path.csv', 'w') as f:
    neg_empi_fin_df['relative_dcm_path'].to_csv(f, index=False)

In [10]:
%%bash -s "$image_dcm_path" "{data_path}negative_path.csv"

# Pulling dicom files with AWS CLI (Python API didn't work)
dcm_dest_path="$1"
dcm_paths="$2"
ind=$((1))

tail -n +2 $dcm_paths | head -n 200 | tail -n 100 | while IFS= read -r line; do
    relative_path=$(echo "$line" | awk -v OFS='/' '{$1=$1; print}')
    dcm_name=$(echo "$relative_path" | cut -d '/' -f 3-)
               
    file="${dcm_dest_path}$relative_path"
    dir=$(dirname "${dcm_dest_path}$dcm_name")
    echo "$ind / 1000"
    # echo "$file"
    # echo "${dcm_dest_path}$dcm_name"
    # echo "$dir"
    if [ -f "${dcm_dest_path}$dcm_name" ]; then
        echo "Moving file $file"
        mkdir -p $(dirname "$file") & mv "${dcm_dest_path}$dcm_name" "${dcm_dest_path}$relative_path"
        rmdir $dir
    else        
        if [ -f "$file" ]; then
            echo "File already present"
            rmdir $dir
        else
            echo "Pulling file $file"
            aws s3 cp "s3://embed-dataset-open/images/$relative_path" "${dcm_dest_path}/$dcm_name" --profile my-dev-profile
        fi
    fi

    ind=$((ind+1))
    clear
done

1 / 1000
File already present


rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.849.113973.3.60.1.62229455.20200820.1142447/1.2.841.113686.2750824660.1597905726.4609.53499': No such file or directory


2 / 1000
File already present


rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.844.113976.3.63.1.60684188.20191210.1133601/1.2.849.113682.2750824547.1575268587.4755.245082': No such file or directory


3 / 1000
File already present


...

rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.848.113977.3.66.1.60984497.20200116.1154344/1.2.847.113686.2750824498.1578983976.4480.257677': No such file or directory


99 / 1000
File already present


rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.843.113975.3.61.1.60965565.20200119.1081928/1.2.846.113689.2750828066.1578895705.4947.18632': No such file or directory


100 / 1000
File already present


rmdir: failed to remove '/media/szelesteya/F824D4D024D492CC/EMBED-images/dicom-negative/1.2.849.113970.3.65.1.60588064.20191118.1071251/1.2.848.113686.2750828054.1574059427.2781.67880': No such file or directory


In [11]:
# Rescale the intensity of the image to get heterogene images with the bit depth of 14
def rescale_to_8bit(image_array):
    upper_percentile = np.percentile(image_array.flatten(), 98) # original_max = np.max(image_array)
    lower_percentile = np.percentile(image_array.flatten(), 2) # original_min = np.min(image_array)
    # max_on_14bit = 16383
    max = 255
    rescaled_array = (image_array - lower_percentile) / (upper_percentile - lower_percentile)
    rescaled_array[rescaled_array < 0] = 0
    rescaled_array[rescaled_array > 1] = 1
    # rescaled_array = np.round((image_array - original_min) / (original_max - original_min) * max_on_14bit).astype(int)
    return np.round(rescaled_array * 255).astype(np.uint8)

# Save DICOM pixel array as PNG
def save_dcm_image_as_png(image, png_filename, bitdepth=8):
    with open(png_filename, 'wb') as f:
        rescaled = rescale_to_8bit(image)
        writer = png.Writer(height=rescaled.shape[0], 
                            width=rescaled.shape[1], 
                            bitdepth=bitdepth, 
                            greyscale=True)
        writer.write(f, rescaled.tolist())

def generate_png_path(empi_anon, png_dir):
    # Get new file name
    new_fn = f"{empi_anon}_neg_conv.png"
    return f'{png_dir}/{new_fn}'

# Convert list of DICOMs to PNGs
def process_dcm_list(dcm_list, png_list):    
    for i, dcm_path in enumerate(dcm_list):    
        if not Path(png_list[i]).exists():
            print(f"Processing DICOM #{i}...")
            
            # Load DICOM
            dcm = pydicom.dcmread(dcm_path)
            img = dcm.pixel_array
            
            # Save PNG            
            save_dcm_image_as_png(img, png_list[i])

def extract_images(data_file_name, dcm_dir, png_dir):
    # Provide a list of DICOM paths and a target directory
    dcm_list = []
    df = pd.read_csv(data_file_name)
    
    for index, row in df.iterrows():
        path = dcm_dir  + row['relative_dcm_path']
        if Path(path).exists():
            dcm_list.append(path)
        
    # Insert png path
    df.loc[:,'png_path'] = df['empi_anon'].apply(lambda x: generate_png_path(x, png_dir))

    # Convert DICOMs
    process_dcm_list(dcm_list[0:300], df['png_path'])

    return df

In [12]:
neg_img_emp = extract_images(data_path + 'negative_empirical.csv', image_dcm_path, image_png_path)
# negative['intensity'] = calculate_intensity_of_png(negative)

with open(data_path + 'negative_empirical_png.csv', 'w') as f:
    (neg_img_emp)[['empi_anon',
                'acc_anon',
                'side',
                'age_at_study',
                'relative_dcm_path',
                'calc_find',
                'calc_distrib',
                'other_find',
                'num_find',
                'eth_desc',
                'study_date_anon',
                'png_path']].to_csv(f, index=False)

Processing DICOM #63...


In [14]:
neg_img_emp[~neg_img_emp['other_find'].notna()]


,empi_anon,acc_anon,side,asses,age_at_study,relative_dcm_path,calc_find,calc_distrib,other_find,num_find,ROI_coords,eth_desc,study_date_anon,png_path
0,29011563,1572834723540517,R,N,82.651937,cohort_1/29011563/1.2.846.113979.3.61.1.622904...,NaN,NaN,NaN,1,(),African American or Black,2021-02-17,/media/szelesteya/F824D4D024D492CC/EMBED-image...
1,11485524,6356263291812846,L,N,74.038481,cohort_1/11485524/1.2.844.113973.3.64.1.622065...,NaN,NaN,NaN,1,(),African American or Black,2021-02-10,/media/szelesteya/F824D4D024D492CC/EMBED-image...
2,41042249,9346948292263998,R,N,67.763198,cohort_2/41042249/1.2.845.113978.3.60.1.621182...,NaN,NaN,NaN,1,(),Caucasian or White,2021-01-31,/media/szelesteya/F824D4D024D492CC/EMBED-image...
3,12927057,6114291372605113,R,N,68.045203,cohort_2/12927057/1.2.841.113975.3.61.1.622750...,NaN,NaN,NaN,1,(),Caucasian or White,2021-01-24,/media/szelesteya/F824D4D024D492CC/EMBED-image...
4,60191895,9641409346677793,R,N,62.073828,cohort_1/60191895/1.2.843.113978.3.64.1.621798...,NaN,NaN,NaN,1,(),African American or Black,2021-01-23,/media/szelesteya/F824D4D024D492CC/EMBED-image...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1941,10412436,4784297769562945,R,N,80.519107,cohort_2/10412436/1.2.842.113970.3.61.1.473439...,NaN,NaN,NaN,1,(),Caucasian or White,2012-09-19,/media/szelesteya/F824D4D024D492CC/EMBED-image...
1942,72504682,7661551911935583,R,N,57.318083,cohort_2/72504682/1.2.847.113977.3.66.1.474765...,NaN,NaN,NaN,1,(),Asian,2012-09-18,/media/szelesteya/F824D4D024D492CC/EMBED-image...
1943,79148688,5277430237922202,L,N,56.923825,cohort_2/79148688/1.2.847.113976.3.62.1.473578...,NaN,NaN,NaN,1,(),Caucasian or White,2012-08-26,/media/szelesteya/F824D4D024D492CC/EMBED-image...
1944,60319885,1310615916903458,R,N,59.650780,cohort_2/60319885/1.2.849.113976.3.64.1.474447...,NaN,NaN,NaN,1,(),African American or Black,2012-08-23,/media/szelesteya/F824D4D024D492CC/EMBED-image...
